In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [24]:
# import the necessary modules
import numpy as np
from sklearn.neural_network import MLPClassifier

# define the number of inputs, hidden nodes, and output
num_inputs = 3
num_hidden = 4
num_output = 1

# initialize the multi-layer perceptron
mlp = MLPClassifier(hidden_layer_sizes=(num_hidden,), activation="logistic")

# define the input and output data
input_data = np.array([[1, 0, 1],
                       [0, 1, 0],
                       [1, 1, 1],
                       [0, 0, 0]])

output_data = np.array([1, 0, 1, 0])

# train the multi-layer perceptron
mlp.fit(input_data, output_data)

# use the trained multi-layer perceptron to make predictions
predictions = mlp.predict(input_data)

# print the predictions
print(predictions)


[1 1 1 0]


/home/mwarren/Projects/GoblinBet/Goblin_Bot/.venv/lib64/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [46]:
# Import data
dfWin = pd.read_csv("./Winners.csv", index_col=0)
dfLose = pd.read_csv("./Losers.csv", index_col=0)
dfLose.head()

,name,desc,cr,immunities,resists,conditions,wins,actions,hp,str,dex,con,int,wis,cha,ac,spd
0,Vrock,Large Fiend Chaotic Evil,6,Poisoned,"Bludgeoning,Piercing,Slashing,Cold,Fire,Lightn...","Green Half-Dragon,Dead",NaN,"Beak: +5, 2d6+3 Piercing\tTalons: +5, 2d10+3 S...",15,17,15,18,8,13,8,15,30
1,Medusa,Medium Monstrosity Lawful Evil,6,NaN,NaN,"Boots of Charging,Dead",NaN,"Multiattack: Snake Hair, 2 Shortsword, Petrify...",70,10,15,16,12,13,15,15,30
2,Shield Guardian,Large Construct Unaligned,7,"Poison,Charmed,Exhaustion,Frightened,Paralyzed...",NaN,"Blessed By God,Regeneration,Stored Spell,Dead",NaN,"Fist: +6, 2d6+4 Bludgeoning\tMultiattack: 2 Fi...",63,18,8,18,7,10,3,17,30
3,Spirit Naga,Large Monstrosity Chaotic Evil,8,"Poison,Poisoned,Charmed",NaN,"Power Attacker,Asleep",NaN,"Bite: +1, 10ft, 1d6+24 Piercing, 7d8 Poison (C...",37,18,17,14,16,15,16,15,40
4,Tyrannosaurus Rex,Huge Beast Unaligned,8,NaN,NaN,"Always Reckless,On Fire,Dead",NaN,"Bite: +9, 10ft, 4d12+7 Piercing, Grappled & Re...",80,25,10,19,2,12,9,13,50


I forgot to put which side the winner and loser were on, therefore I need to randomize where to put the winning creature and where to put the losing creature.
This will allow the model from not constantly predicting the creature on a specific side

In [92]:
import random
# Returns all stat values of a dataframe as a 2d array
def getStatsValues(df: pd.DataFrame):
    return df.loc[:, ["hp", "str", "dex", "con", "int", "wis", "cha", "ac", "spd"]].values

# Get Input values
dfWinStats = getStatsValues(dfWin)
dfLoseStats = getStatsValues(dfLose)
# Append input values together
dfStats = np.concatenate([dfWinStats, dfLoseStats], axis=1)
# Generate random list representing which side won and which side lost
inputY = np.random.randint(low=0,high=2,size=dfStats.shape[0])


data = []
# Add
for i in range(dfWinStats.shape[0]):
    if inputY[i]:
        res = np.concatenate([dfWinStats[i], dfLoseStats[i]])
    else:
        res = np.concatenate([dfLoseStats[i], dfWinStats[i]])
    data.append(res)

In [93]:
# Get the distribution of target values

one = 0
zero = 0

for val in inputY:
    if val:
        one += 1
    else:
        zero += 1

print(f"One: {one}")
print(f"Zero: {zero}")

One: 645
Zero: 666


In [145]:
splitSize = 0.9




X_train, X_test, y_train, y_test = train_test_split(data, inputY, test_size=(1-splitSize), random_state=47)


In [146]:
print(len(X_test[0]))

18


In [147]:
clf = MLPClassifier(hidden_layer_sizes=(60, 20), max_iter=100000, learning_rate_init=0.00001)

clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

In [148]:
from sklearn.metrics import confusion_matrix, classification_report


print(classification_report(y_test, predictions))



              precision    recall  f1-score   support

           0       0.61      0.67      0.64        67
           1       0.62      0.55      0.59        65

    accuracy                           0.61       132
   macro avg       0.61      0.61      0.61       132
weighted avg       0.61      0.61      0.61       132

